In [1]:
mp_pdf = True
import sys, os
if mp_pdf:
    extension = "pdf"
    import matplotlib
    matplotlib.use('pgf')
    pgf_with_latex = {
        "text.usetex": True,            # use LaTeX to write all text
        "pgf.rcfonts": False,           # Ignore Matplotlibrc
        "text.latex.unicode": True,
        "pgf.preamble": [
            #r"\usepackage[utf8x]{inputenc}",
            r"\usepackage{xcolor}"
        ],
        "pgf.texsystem" : "xelatex",
        "figure.figsize": [12,7]
    }
    matplotlib.rcParams.update(pgf_with_latex)
else:
    extension = "png"
    
import matplotlib as mpl



dsize = 7.0866142
run_id=1861
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from adjustText import adjust_text, get_renderer, get_bboxes
import operator

mpl.rcParams['font.size'] = 6

/home/max/.local/lib/python3.6/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


In [2]:
mdf = pd.read_csv(f'../tables/newness_representation_{run_id}.csv')
mdf.head()

mdf[mdf['primary_wg']==3].head()

,ipcc_coverage,ipcc_score,ipcc_share,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av,lrep,x_q,y_q
1,0.143624,17.853955,0.004963,3,199.901053,0.005426,"sector, sectoral emissions",-0.000463,0.000463,0.914610,0.542141,4.593220,-0.089258,7,5
6,0.068143,6.868519,0.001909,3,169.577758,0.004603,"electricity, electricity generation & demand",-0.002694,0.002694,0.414773,0.535850,4.620690,-0.880023,7,1
14,0.086559,16.772689,0.004662,3,323.258870,0.008775,"technology, technology developement & carbon c...",-0.004112,0.004112,0.531335,0.536115,4.620690,-0.632362,7,2
22,0.183268,39.718230,0.011040,3,343.127380,0.009314,"countries, country analysis and comparisons",0.001726,0.001726,1.185362,0.530242,4.508475,0.170048,5,7
23,0.137934,22.423120,0.006233,3,263.391556,0.007150,"prices, (CO2) prices and markets",-0.000917,0.000917,0.871788,0.535281,4.603448,-0.137209,7,5


In [3]:
topic_dis_df = pd.read_csv(f'../tables/topic_discipline_{run_id}.csv')
topic_dis_df = topic_dis_df.rename(columns={"dis_share":"dis_score"})

topic_dis_df['dis_total'] = topic_dis_df.groupby('title')['dis_score'].transform('sum')

topic_dis_df['dis_share'] = topic_dis_df['dis_score'] / topic_dis_df['dis_total']

topic_dis_df.head()

,dis_score,doctopic__doc__wc__oecd,primary_wg,title,dis_share_pre,dis_share_post,dis_total,dis_share
0,98.609781,Agricultural Sciences,2,"indices, indices, correlation and relationships",45.168047,53.441733,1028.065952,0.095918
1,185.606297,Engineering and Technology,2,"indices, indices, correlation and relationships",73.102808,112.503490,1028.065952,0.180539
2,2.184284,Humanities,2,"indices, indices, correlation and relationships",0.756985,1.427299,1028.065952,0.002125
3,13.903700,Medical and Health Sciences,2,"indices, indices, correlation and relationships",5.620609,8.283041,1028.065952,0.013524
4,655.278080,Natural Sciences,2,"indices, indices, correlation and relationships",310.473621,344.804458,1028.065952,0.637389


In [4]:
topic_dis_wide = topic_dis_df.pivot_table(
    index=["title"],columns="doctopic__doc__wc__oecd",values="dis_share"
).reset_index()

topic_dis_wide.head()

doctopic__doc__wc__oecd,title,Agricultural Sciences,Engineering and Technology,Humanities,Medical and Health Sciences,Natural Sciences,Social Sciences
0,"(e)-vehicles, electric, hybrid and combustion ...",0.017059,0.569020,0.002428,0.006232,0.247428,0.154439
1,"CH4, CH4",0.204914,0.166121,0.000209,0.003133,0.618849,0.006607
2,"CLC, chemical looping combustion (CLC)",0.035575,0.512716,0.001255,0.004690,0.426735,0.017448
3,"CO2 storage, geological CO2 storage",0.047830,0.484719,0.001622,0.004390,0.434192,0.026446
4,"CO2, CO2",0.084190,0.363998,0.000726,0.002875,0.527574,0.019829


In [5]:
df = pd.concat([mdf.sort_values('title').reset_index(drop=True),topic_dis_wide.sort_values('title').reset_index(drop=True)],axis=1)
df = pd.merge(mdf, topic_dis_wide)
df.head()

,ipcc_coverage,ipcc_score,ipcc_share,primary_wg,score,share,title,deviation,abs_md,representation,...,year_av,lrep,x_q,y_q,Agricultural Sciences,Engineering and Technology,Humanities,Medical and Health Sciences,Natural Sciences,Social Sciences
0,0.127550,16.243344,0.004515,1,191.274690,0.005192,"transport, transport",-0.000677,0.000677,0.869630,...,4.446154,-0.139688,3,5,0.034290,0.286544,0.002646,0.008167,0.527715,0.138370
1,0.143624,17.853955,0.004963,3,199.901053,0.005426,"sector, sectoral emissions",-0.000463,0.000463,0.914610,...,4.593220,-0.089258,7,5,0.055740,0.393644,0.005167,0.007054,0.325992,0.208360
2,0.275522,44.160043,0.012275,1,207.308023,0.005627,"uncertainty, uncertainty & estimation",0.006648,0.006648,2.181373,...,4.521739,0.779954,5,9,0.069837,0.200647,0.004018,0.007810,0.632479,0.084286
3,0.127781,22.153958,0.006158,1,224.378004,0.006091,"air pollution, air pollution",0.000068,0.000068,1.011085,...,4.402778,0.011024,2,6,0.072704,0.295369,0.003491,0.019946,0.548198,0.058452
4,0.143087,26.783303,0.007445,2,231.589446,0.006286,"humans, human activity, human behaviour & impa...",0.001159,0.001159,1.184300,...,4.527027,0.169152,5,7,0.065904,0.155102,0.017031,0.027815,0.621609,0.111205


In [75]:
pdf = df[df['primary_wg']==3]

fig, ax = plt.subplots(figsize=(dsize*0.5,dsize*0.3), dpi=200)

def wg_socsci_rep(ax, pdf, c):
    ax.scatter(
        pdf['Social Sciences'], 
        pdf['lrep'],
        c=c,
        linewidths=1,
        edgecolors="grey",
        s=5
    )
    texts = []
    for i, row in pdf.iterrows():
        stext = row['title'].split(',')[0].replace('{','')
        texts.append(ax.text(row['Social Sciences'], row['lrep'], stext, fontsize=6))

    ax.axhline(0,c="grey",lw=1,alpha=0.8,linestyle=":",zorder=-50)
    ax.set_xlabel("Social science proportion")
    ax.set_ylabel("IPCC representation (log)")

    adjust_text(texts, ax=ax)
    
wg_socsci_rep(ax, pdf, "#8da0cb")

fig.patch.set_facecolor('white') 

#fig.tight_layout()

plt.savefig(
    f'../plots/run_{run_id}_wg3_socsci.{extension}',
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)
plt.show()

/home/max/.local/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/max/.local/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [7]:
fig, axs = plt.subplots(3,figsize=(dsize,dsize),dpi=200)
colors=["#66c2a5","#fc8d62","#8da0cb"]
for wg in [1,2,3]:
    
    ax = axs[wg-1]

    pdf = df[df['primary_wg']==wg]
    c = colors[wg-1]
    wg_socsci_rep(ax,pdf,c)
    
fig.tight_layout()

plt.savefig(
    f'../plots/run_{run_id}_wgs_socsci.{extension}',
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

/home/max/.local/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [88]:
fig, ax = plt.subplots(figsize=(dsize*0.5,dsize*0.3),dpi=200)
import textwrap as tw


oecd_df_ys = pd.read_csv(f'../tables/oecd_representation_{run_id}.csv')
fdf = oecd_df_ys[oecd_df_ys['ar']>1]

fdf['ar_share_total'] = fdf.groupby('ar')['share'].transform('sum')

size = 25

cdict = {
    'Natural Sciences': '#0072B2',
    'Engineering and Technology': '#CC79A7',
    'Agricultural Sciences': '#009E73',
    'Social Sciences': '#E69F00',
    'Medical and Health Sciences': '#F0E442',
    'Humanities': '#000000',
    None: '#F0F0F0'
}

def simplified_oecd_rep(ax, fdf):
    ticks = []
    labels = []
    
    w = 13

    for i, (name, group) in enumerate(fdf.sort_values('total_share').groupby('wc__oecd', sort=False)):
        color = cdict[name]
        group = group.replace(-np.inf, np.nan).dropna().sort_values('ar').reset_index(drop=True)
        name = f"{name} ({group['total_share'][0]:.1%})"
        name = name.replace(' and ',' \& ')
        name = tw.fill(tw.dedent(name),width=w)
        ticks.append(i)
        labels.append(name)
        #color = group.reset_index()['color'][0]
        
        group['s_share'] = group['share'] / group['ar_share_total']

        start = group['lrep'][0]
        end = group['lrep'][len(group)-1]
        ax.plot(
            [i,i],
            [start,end],
            color="grey",
            zorder=-50
        )
        ax.scatter(
            i,
            start,
            label=None,
            color=color,
            s = size,
            edgecolor="grey",
            #marker="x"
        )
        ax.scatter(
            i,
            end,
            label=None,
            color=color,
            s = size,
            edgecolor="grey",
            marker="^"
        )


    ax.set_xticks(ticks)
    ax.set_xticklabels(labels, ha="center", va="top", rotation=0, fontsize=5.8)
    #ax.set_xticklabels(labels, ha="right", va="top", rotation=25)
    ax.axhline(0,c="grey",lw=1,alpha=0.8,linestyle=":",zorder=-50)

    for l in [("AR2","o"),("AR5","^")]:
        ax.scatter(
            [],[],
            c='#f0f0f0',s=size,
            edgecolor="grey",
            label=l[0],marker=l[1]
        )

    ax.legend()
    
    ax.set_xlabel("Discipline")
    ax.set_ylabel("IPCC representation (log)")
    
    
simplified_oecd_rep(ax, fdf)

fig.patch.set_facecolor('white') 


plt.savefig(
    '../plots/ipcc_representation/ipcc_rep_oecds_simplified.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)


plt.show()

/home/max/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/max/.local/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [9]:

#fig, axs = plt.subplots(2,1,figsize=(dsize*0.5,dsize*0.7), dpi=200)
#fig, axs = plt.subplots(1,2,figsize=(dsize*0.5,dsize*0.7))
fig, axs = plt.subplots(1,2,figsize=(dsize,dsize*0.4))#,sharey=True)


simplified_oecd_rep(axs[0], fdf)

wg_socsci_rep(axs[1], pdf, "#8da0cb")

axs[1].set_ylabel(None)
axs[1].set_yticks([])

ymin = min(a.get_ylim()[0] for a in axs)
ymax = max(a.get_ylim()[1] for a in axs)

for a in axs:
    a.set_ylim((ymin,ymax))

#axs[0].set_ylim(axs[1].get_ylim())

fig.patch.set_facecolor('white') 
fig.tight_layout()

plt.savefig(
    '../plots_pub/panel_representation.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

/home/max/.local/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [85]:
def rep_time(ax, mdf, lquads=False, log_ticks=False):
    x = 'year_av'
    y = 'representation'
    y = 'lrep'

    mdf['lrep'] = np.log(mdf['representation'])

    cmap = {1: "#66c2a5", 2: "#fc8d62" , 3: "#8da0cb"}
    colors = [cmap[i] for i in mdf['primary_wg']]


    mdf.plot.scatter(
        x,y,s=mdf['score']*0.03,
        c = colors,ax=ax
    )
    
    for k,v in cmap.items():
        ax.scatter(
            [],[],
            c=v,s=size,
            edgecolor="grey",
            label=f"WG {k}",
        )   
        
    ax.legend()


    # Fit a trendline
    #z = np.polyfit(x=mdf[x], y=mdf[y], deg=1)
    #p = np.poly1d(z)
    #mdf['trendline'] = p(mdf.loc[:, x])
    #plt.plot(mdf[x],mdf['trendline'])


    # Quantile x and y
    qs = 10
    mdf['x_q'] = pd.qcut(mdf[x], qs, labels=False)
    mdf['y_q'] = pd.qcut(mdf[y], qs, labels=False)
    texts = []

    # write texts for dots in outer quantiles
    for i, row in mdf[(mdf['x_q'].isin([0,qs-1])) | (mdf['y_q'].isin([0,qs-1]))].iterrows(): 
        t = row['title'].split(',')[0].strip('{} ')
        texts.append(
            ax.text(row[x],row[y],t,ha='center', va='center',fontsize=6)
        )

    # Tweak the labels to avoid overlap    
    adjust_text(
        texts, 
        arrowprops=dict(arrowstyle="-", color='black', lw=0.5)
    )


    # Split the graph into quadrants
    ymid = 0
    ax.axhline(ymid,c="grey",lw=1,alpha=0.8,linestyle=":",zorder=-50)
    ax.axvline(mdf[x].median(),c="grey",lw=1,alpha=0.8,linestyle=":",zorder=-50)



    # Put labels on the quadrants
    # x
    d1 = mdf[x].median()-plt.xlim()[0]
    d2 = mdf[x].median()-plt.xlim()[1]

    xlabpoints = [mdf[x].median()-d1/2,mdf[x].median()-d2/2]
    
    xrange = mdf[x].max()-mdf[x].min()
    xlabpoints = [mdf[x].min()+xrange*0.15,mdf[x].min()+xrange*0.85]
    
    pad = (mdf[x].min() - ax.get_xlim()[0])*0.25
    
    xlabpoints = [ax.get_xlim()[0] + pad, mdf[x].median() + pad]
    
    if lquads:
        labels = [
            ['Under-represented,\nolder topics','Under-represented,\nnewer topics'],
            ['Over-represented,\nolder topics','Over-represented,\nnewer topics']
        ]

        pylims = plt.ylim()

        tpad = (pylims[1]-pylims[0])*0.13
        
        tpad=-pad*12

        ops = [operator.lt,operator.gt]

        for i in [0,1]:
            for j in [0,1]:
                if j==0:
                    tpadx=tpad*-1
                else:
                    tpadx=tpad*1


                q = mdf[(ops[i](mdf[x],mdf[x].median())) & (ops[j](mdf[y],ymid))]
                qshare = q['score'].sum()/mdf['score'].sum()
                l = labels[j][i].replace("\n"," ")
                ax.text(
                    xlabpoints[i],
                    pylims[j]+tpadx,
                    f"{l} {qshare:.0%}",
                    va="center",ha="left",
                    fontweight='bold',
                    #bbox={'facecolor':'red', 'alpha':qshare*2-0.1, 'pad':6},
                    fontsize=7
                )

    if log_ticks:
        fmax = None
        fmin = None
        rmax = mdf['lrep'].max()
        if fmax:
            rmax = fmax
        rmin = mdf['lrep'].min()
        if fmin:
            rmin = -fmin

        if rmin > -1:
            rmin = -1
        if np.exp(rmax) < 2:
            rmax = np.log(2.1)

        lmin = int(1/np.exp(rmin)//1)*-1
        lmax = int(np.exp(rmax)//1)



        ytick_labels = [i for i in range(lmin,lmax+1) if i not in [-1,1]]
        yticks = []
        for x in ytick_labels:
            if x < 0:
                y = np.log(-1/x)
            elif x==0:
                y = 0
            else:
                y = np.log(x)
            yticks.append(y)

        ax.set_yticks(yticks)
        ax.set_yticklabels(ytick_labels)

    #ax.get_yaxis().set_visible(False)
    #ax.tick_params(axis=u'both', which=u'both',length=0)
    #ax2.tick_params(axis=u'both', which=u'both',length=0)
    ax.set_xlabel('Assessment period\noccurence')
    ax.set_ylabel('IPCC representation (log)')

fig, ax = plt.subplots(figsize=(dsize,dsize*0.62))

rep_time(ax, mdf, lquads=True)

plt.tight_layout(h_pad=100)

fig.patch.set_facecolor('white')    


plt.savefig(
    '../plots_pub/rep_time.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

/home/max/.local/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/max/.local/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [89]:
fig = plt.figure(figsize=(dsize, dsize*1.1), dpi=200)


gsize = (8,2)

ax1 = plt.subplot2grid(gsize, (0, 0), colspan=1, rowspan=3)

ax2 = plt.subplot2grid(gsize, (0, 1), colspan=1, rowspan=3)

simplified_oecd_rep(ax1, fdf)

wg_socsci_rep(ax2, pdf, "#8da0cb")

ax2.set_ylabel(None)
ax2.set_yticks([])

axs = [ax1,ax2]

ymin = min(a.get_ylim()[0] for a in axs)
ymax = max(a.get_ylim()[1] for a in axs)

for a in axs:
    a.set_ylim((ymin,ymax))

ax3 = plt.subplot2grid(gsize, (3, 0), colspan=2, rowspan=5)

ax1.set_title(
    "a) IPCC representation of disciplines", 
    fontsize=7, fontweight="bold",loc="left"
)

ax2.set_title(
    "b) Social science proportion and IPCC representation of WG 3 topics", 
    fontsize=7, fontweight="bold",loc="left"
)

ax3.set_title(
    "c) Topic novelty and representation", 
    fontsize=7, fontweight="bold",loc="left"
)

rep_time(ax3, mdf, lquads=True)

fig.canvas.draw()

fig.tight_layout()

fig.patch.set_facecolor('white')    

plt.savefig(
    '../plots_pub/big_panel_representation.{}'.format(extension),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

/home/max/.local/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/max/.local/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
